In [1]:
from prediction_utils.extraction_utils.database import BQDatabase

In [2]:
db = BQDatabase()

/share/pi/nigam/spfohl/conda_envs/py_env/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [13]:
df = db.read_sql_query('SELECT visit_start_datetime from starr_omop_cdm5_deid_20200404.visit_occurrence LIMIT 10', progress_bar_type=None)

Downloading: 100%|██████████| 10/10 [00:00<00:00, 13.65rows/s]


In [5]:
# db.to_sql(
#     df,
#     destination_table='plp_cohort_tables.temp',
# )

1it [00:05,  5.11s/it]


In [10]:
param_dict = {
    'database': 'som-nero-phi-nigam-starr',
    'rs_schema': 'plp_cohort_tables',
    'cohort_name': 'admission_rollup_20200404'
}
param_dict['cohort_name_sampled'] = '{}_sampled'.format(param_dict['cohort_name'])

In [11]:
sample_query = """
    CREATE OR REPLACE TABLE {database}.{rs_schema}.{cohort_name_sampled} AS
    SELECT * EXCEPT (rnd, pos)
    FROM (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY person_id ORDER BY rnd) AS pos
        FROM (
            SELECT 
                *,
                FARM_FINGERPRINT(CONCAT(CAST(person_id AS STRING), CAST(admit_date AS STRING), CAST(discharge_date AS STRING))) as rnd
            FROM {rs_schema}.{cohort_name}
        )
    )
    WHERE pos = 1
""".format_map(param_dict)

'\n    CREATE OR REPLACE TABLE som-nero-phi-nigam-starr.plp_cohort_tables.admission_rollup_20200404_sampled AS\n    SELECT * EXCEPT (rnd, pos)\n    FROM (\n        SELECT *, ROW_NUMBER() OVER(PARTITION BY person_id ORDER BY rnd) AS pos\n        FROM (\n            SELECT \n                *,\n                FARM_FINGERPRINT(CONCAT(CAST(person_id AS STRING), CAST(admit_date AS STRING), CAST(discharge_date AS STRING))) as rnd\n            FROM plp_cohort_tables.admission_rollup_20200404\n        )\n    )\n    WHERE pos = 1\n'

In [12]:
db.execute_sql(sample_query)